Diego - Last time I ate tacos like a month ago and I play videogames to pass time

## Workshop - Regression-Based Classification

Does `statsmodels` marginal effect use the average of covariates or the average predicted values? 
- Use the class data.
- Show your work.

Load the necessary packages and data:

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm # progress bar

import statsmodels.api as sm
from sklearn import linear_model as lm

from sklearn.model_selection import GridSearchCV, train_test_split, KFold
from sklearn.metrics import confusion_matrix, plot_confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc = {'axes.titlesize': 24,
             'axes.labelsize': 20,
             'xtick.labelsize': 12,
             'ytick.labelsize': 12,
             'figure.figsize': (8, 4.5)})
sns.set_style("white") # for plot_confusion_matrix()

In [3]:
df = pd.read_pickle("C:/Users/dp846/OneDrive/Desktop/ECON490ML/class data/class_data.pkl")
df_prepped = df.drop(columns = ['urate_bin', 'year']).join([
    pd.get_dummies(df['urate_bin'], drop_first = True),
    pd.get_dummies(df.year, drop_first = True)    
])
y = df_prepped['pos_net_jobs'].astype(float)
x = df_prepped.drop(columns = 'pos_net_jobs')

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 2/3, random_state = 490)

x_train_std = x_train.apply(lambda x: (x - np.mean(x))/np.std(x), axis = 0)
x_test_std  = x_test.apply(lambda x: (x - np.mean(x))/np.std(x), axis = 0)

x_train_std = sm.add_constant(x_train_std)
x_test_std  = sm.add_constant(x_test_std)
x_train     = sm.add_constant(x_train)
x_test      = sm.add_constant(x_test)


Fit a logistic regression using either `sm.Logit()` or `smf.logit()`.

In [4]:
fit_logit = sm.Logit(y_train, x_train).fit()

Optimization terminated successfully.
         Current function value: 0.599666
         Iterations 6


Get the marginal effects (`.get_margeff()`). Print the summary (`.summary()`).

In [5]:
fit_logit.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                          Results: Logit
==================================================================
Model:               Logit            Pseudo R-squared: 0.126     
Dependent Variable:  pos_net_jobs     AIC:              40700.1748
Date:                2021-03-02 14:48 BIC:              40936.2385
No. Observations:    33889            Log-Likelihood:   -20322.   
Df Model:            27               LL-Null:          -23242.   
Df Residuals:        33861            LLR p-value:      0.0000    
Converged:           1.0000           Scale:            1.0000    
No. Iterations:      6.0000                                       
------------------------------------------------------------------
                   Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
------------------------------------------------------------------
const             -1.9804   0.1454 -13.6244 0.0000 -2.2653 -1.6955
pct_d_rgdp         0.0159   0.0014  11.0808 0.0000  0.0131  0.0187
emp_estabs         0.0383   0.0029  13.2672 0.0000  0.0327  0.0440
estabs_entry_rate  0.1938   0.0054  35.7595 0.0000  0.1832  0.2045
estabs_exit_rate  -0.1625   0.0060 -27.1367 0.0000 -0.1743 -0.1508
pop                0.0000   0.0000   2.7815 0.0054  0.0000  0.0000
pop_pct_black     -0.0037   0.0009  -3.9538 0.0001 -0.0055 -0.0019
pop_pct_hisp       0.0065   0.0010   6.6630 0.0000  0.0046  0.0084
lfpr               0.0014   0.0014   1.0370 0.2997 -0.0012  0.0041
density            0.0000   0.0000   0.8976 0.3694 -0.0000  0.0000
lower              0.3427   0.0335  10.2204 0.0000  0.2770  0.4084
similar            0.1708   0.0357   4.7822 0.0000  0.1008  0.2408
2003               1.0795   0.0685  15.7513 0.0000  0.9451  1.2138
2004               1.2705   0.0701  18.1212 0.0000  1.1330  1.4079
2005               1.2782   0.0718  17.7988 0.0000  1.1374  1.4189
2006               1.7408   0.0752  23.1429 0.0000  1.5934  1.8882
2007               0.5726   0.0679   8.4340 0.0000  0.4396  0.7057
2008               0.9002   0.0689  13.0671 0.0000  0.7652  1.0353
2009              -0.5546   0.0813  -6.8221 0.0000 -0.7139 -0.3953
2010               0.1881   0.0712   2.6427 0.0082  0.0486  0.3275
2011               1.2499   0.0694  18.0167 0.0000  1.1139  1.3858
2012               1.5995   0.0719  22.2615 0.0000  1.4587  1.7404
2013               1.2136   0.0701  17.3026 0.0000  1.0762  1.3511
2014               1.5303   0.0716  21.3646 0.0000  1.3899  1.6707
2015               1.6389   0.0738  22.2195 0.0000  1.4944  1.7835
2016               1.2858   0.0714  18.0141 0.0000  1.1459  1.4257
2017               1.1100   0.0703  15.7814 0.0000  0.9721  1.2478
2018               1.4769   0.0726  20.3342 0.0000  1.3346  1.6193
==================================================================

"""

***
# Covariate Averages
$$
\frac{\partial p(x_i)}{\partial \beta_1} \approx \frac{e^{\hat{\beta}_0 + \bar{x}\hat{\beta}_1 + \bar{x}\hat{\beta_2}}}{(1 + e^{\hat{\beta}_0 + \bar{x}\hat{\beta}_1 + \bar{x}\hat{\beta_2}})^2}\hat{\beta}
$$

***
# Predicted values Averages
$$
\frac{\partial p(x_i)}{\partial \beta_1} \approx \frac{1}{n} \sum_{i=1}
^n \frac{e^{\hat{y}_i}}{1 + e^{\hat{y}_i}}\hat{\beta}
$$

In [ ]:
fit_logit.predict(x_test).describe()

In [6]:
fit_logit.get_margeff().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
        Logit Marginal Effects       
=====================================
Dep. Variable:           pos_net_jobs
Method:                          dydx
At:                           overall
=====================================================================================
                       dy/dx    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
pct_d_rgdp            0.0033      0.000     11.150      0.000       0.003       0.004
emp_estabs            0.0079      0.001     13.394      0.000       0.007       0.009
estabs_entry_rate     0.0401      0.001     38.375      0.000       0.038       0.042
estabs_exit_rate     -0.0336      0.001    -28.236      0.000      -0.036      -0.031
pop                3.029e-08   1.09e-08      2.782      0.005    8.95e-09    5.16e-08
pop_pct_black        -0.0008      0.000     -3.957      0.000      -0.001      -0.000
pop_pct_hisp          0.0013      0.000      6.678      0.000       0.001       0.002
lfpr                  0.0003      0.000      1.037      0.300      -0.000       0.001
density             1.59e-06   1.77e-06      0.898      0.369   -1.88e-06    5.06e-06
lower                 0.0709      0.007     10.278      0.000       0.057       0.084
similar               0.0353      0.007      4.788      0.000       0.021       0.050
2003                  0.2232      0.014     15.973      0.000       0.196       0.251
2004                  0.2627      0.014     18.459      0.000       0.235       0.291
2005                  0.2643      0.015     18.120      0.000       0.236       0.293
2006                  0.3600      0.015     23.839      0.000       0.330       0.390
2007                  0.1184      0.014      8.467      0.000       0.091       0.146
2008                  0.1862      0.014     13.189      0.000       0.159       0.214
2009                 -0.1147      0.017     -6.834      0.000      -0.148      -0.082
2010                  0.0389      0.015      2.644      0.008       0.010       0.068
2011                  0.2585      0.014     18.345      0.000       0.231       0.286
2012                  0.3308      0.014     22.889      0.000       0.302       0.359
2013                  0.2510      0.014     17.594      0.000       0.223       0.279
2014                  0.3165      0.014     21.919      0.000       0.288       0.345
2015                  0.3389      0.015     22.840      0.000       0.310       0.368
2016                  0.2659      0.014     18.345      0.000       0.238       0.294
2017                  0.2295      0.014     16.001      0.000       0.201       0.258
2018                  0.3054      0.015     20.812      0.000       0.277       0.334
=====================================================================================
"""

*** 
# Interpretation

Interpret the marginal effect on one feature.